In [65]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [66]:
pickle_file = '/Users/rringham/Projects/TensorFlow/tensorflow/tensorflow/examples/udacity/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [67]:
image_size = 28
num_labels = 10
num_uns_labels = 50

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_uns_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 50)
Validation set (10000, 784) (10000, 50)
Test set (10000, 784) (10000, 50)


In [68]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Notes
---------

With two hidden layers, the best accuracy I've been able to get on the test set below is roughly 90%:
  - *h1:* 1024 neurons
  - *h2:* 1024 neurons
  - *h3:* 305 neurons
  - *h4:* 75 neurons
  - Each set of weights initialized with **stddev**=sqrt(2/n), where n = # of neurons in previous layer
  - No dropout
 
 ---


In [69]:
import math as math

batch_size = 1
initial_learning_rate_value = 0.5
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_uns_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)    
    
  # learning rate decay
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(initial_learning_rate_value, global_step, 1, 0.9999)

  # Variables
  h1_size = 1024  
  h2_size = 75
    
  weights_h1 = tf.Variable(tf.truncated_normal([image_size * image_size, h1_size], stddev=math.sqrt(2.0/(image_size*image_size))))
  biases_h1 = tf.Variable(tf.zeros([h1_size]))

  weights_h2 = tf.Variable(tf.truncated_normal([h1_size, h2_size], stddev=math.sqrt(2.0/h1_size)))
  biases_h2 = tf.Variable(tf.zeros([h2_size]))
    
  weights_o = tf.Variable(tf.truncated_normal([h2_size, num_uns_labels], stddev=math.sqrt(2.0/h2_size)))
  biases_o = tf.Variable(tf.zeros([num_uns_labels]))  

  # Model.
  def model(data):
    h1 = tf.nn.relu(tf.matmul(data, weights_h1) + biases_h1)
    h2 = tf.nn.relu(tf.matmul(h1, weights_h2) + biases_h2)
    return tf.matmul(h2, weights_o) + biases_o
    
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + ((beta * tf.nn.l2_loss(weights_h1)) + (beta * tf.nn.l2_loss(weights_h2)) + (beta * tf.nn.l2_loss(weights_o)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Predictions for the training, validation, and test data.  
  train_prediction = tf.nn.softmax(model(tf_train_dataset))  
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [105]:
num_steps = 10
num_tsteps = 1
labels_used = 0

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate training data component of minibatch
    batch_data = train_dataset[offset:(offset + batch_size), :]
    actual_labels = train_labels[offset:(offset + batch_size), :]
    print(actual_labels)
    
    # Assign label
    current_label = 0
    if labels_used == 0:
      print("Initializing offset: %d with current label of: %d" % (offset, current_label))
      labels_used += 1
      current_label = 0
    else:
      print("Determining label for offset: %d" % (offset))
      label_pred = session.run([train_prediction], feed_dict={tf_train_dataset : batch_data})
      print(label_pred)
      print(np.argmax(label_pred))
      current_label = np.argmax(label_pred)
    
    # Generate label data component of minibatch
    current_labels = np.array([current_label])
    current_labels_1h = (np.arange(num_uns_labels) == current_labels[:,None]).astype(np.float32)    
    batch_labels = current_labels_1h
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    for tstep in range(num_tsteps):
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
          
      print("    Minibatch loss at step %d, tstep: %d: %f," % (step, tstep, l))
      print("    Learning rate at step %d, tstep: %d: %f" % (step, tstep, learning_rate.eval()))
      print("    Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("    Validation accuracy: %.1f%%\n" % accuracy(valid_prediction.eval(), valid_labels))


#   for step in range(num_steps):
#     # Pick an offset within the training data, which has been randomized.
#     # Note: we could use better randomization across epochs.
#     offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
#     # Generate a minibatch.
#     batch_data = train_dataset[offset:(offset + batch_size), :]
#     batch_labels = train_labels[offset:(offset + batch_size), :]
    
#     # Prepare a dictionary telling the session where to feed the minibatch.
#     # The key of the dictionary is the placeholder node of the graph to be fed,
#     # and the value is the numpy array to feed to it.
#     feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
#     _, l, predictions = session.run(
#       [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
#     if (step % 500 == 0):
#       print("Minibatch loss at step %d: %f" % (step, l))
#       print("Learning rate at step %d: %f" % (step, learning_rate.eval()))
#       print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
#       print("Validation accuracy: %.1f%%\n" % accuracy(
#         valid_prediction.eval(), valid_labels))
        
#     if (step % 2000 == 0):
#       print("Test accuracy: %.1f%%\n" % accuracy(test_prediction.eval(), test_labels))
        
  print("Final test accuracy: %.1f%%\n" % accuracy(test_prediction.eval(), test_labels))

Initialized
[[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
Initializing offset: 0 with current label of: 0
    Minibatch loss at step 0, tstep: 0: 12.840612,
    Learning rate at step 0, tstep: 0: 0.499950
    Minibatch accuracy: 0.0%
    Validation accuracy: 10.0%

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
Determining label for offset: 1
[array([[  9.99999881e-01,   2.24249078e-10,   2.47857151e-11,
          1.94666629e-08,   8.71971453e-11,   1.35435940e-09,
          1.45522171e-11,   8.36734571e-10,   1.10039777e-09,
          2.40754687e-08,   3.25361071e-08,   7.68593869e-11,
          1.64139013e-09,   1.73332615e-09,   9.90594606e-10,
          1.32575939e